In [1]:
import numpy as np 
import pandas as pd
from bs4 import BeautifulSoup
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
import tweepy
from keras.models import load_model

Using TensorFlow backend.


## Twitter Pull

In [2]:
def pull_tweets(search_term):

    api_key = "ccUDQyUZqI3ljuvSdpGwnEVtA"
    api_secret = "EnyT1YekZlwIw9drTanRlB3CEXHRnx7YJ1R578HXHH2fFi8nQi"
    access_token = "1153798652290203651-zlVEr2pkOIoR5vccxYSROXrzD0PGsk"
    access_token_secret = "aEZ8f10oyknxpOvtog5D2icjgl3O6s61n5i5o2TavrAoP"

    auth = tweepy.OAuthHandler(api_key, api_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True)
    tweet_list = []
    date_list = []
    print(f'pulling tweets related to {search_term}')
    for tweet in tweepy.Cursor(api.search,q=search_term,tweet_mode='extended',lang="en",since="2018-07-21").items(10):
        if (not tweet.retweeted) and ('RT @' not in tweet.full_text):
            tweet_list.append(tweet.full_text)
            date_list.append(tweet.created_at)

    return {'dates': date_list, 'tweets': tweet_list}

In [3]:
search_term = 'Trump'
tweeter_pull = pull_tweets(search_term)
tweets = tweeter_pull['tweets']

pulling tweets related to Trump


## LSTM Model

In [4]:
# Model params
unique_words = 28701
len_max = 53

In [5]:
def clean_sentences(tweet_input):
    #tweet_input must be a list of tweets


    tweets = []
    for x in tweet_input:
        # remove non-alphabetic characters
        tweet_text = re.sub("[^a-zA-Z]"," ", str(x))
        
        #remove html content
        tweet_text = BeautifulSoup(tweet_text).get_text()
        
        # tokenize
        words = word_tokenize(tweet_text.lower())
        
        # lemmatize each word to its lemma
        lemma_words = [lemmatizer.lemmatize(i) for i in words]
        
        tweets.append(lemma_words)
        
    return(tweets)


In [6]:
def token_maker(cleaned1_tweets):
    # Actual tokenizer of keras and convert to sequences

    tokenizer = Tokenizer(num_words=unique_words)
    tokenizer.fit_on_texts(list(cleaned1_tweets))

    # texts_to_sequences
    # ARGUMENTS: list of texts to turn to sequences
    # RETURN: list of sequences (one per text input)

    return tokenizer.texts_to_sequences(cleaned1_tweets)

In [7]:
def keras_output_sklearn(y):
    
    result = []
    
    for element in y:
        result.append(np.argmax(element))
        

    return result

In [8]:
def all_together_now(search_term):
    print(f'Pulling tweets related to {search_term}')
    og_tweets = pull_tweets(search_term)

    print('Cleaning tweets')
    cleaned1 = clean_sentences(og_tweets['tweets'])

    print('Tokenizing tweets')
    tokened_tweets = token_maker(cleaned1)

    print('Padding tweets')
    padded_tweets = sequence.pad_sequences(tokened_tweets, maxlen=len_max)

    print('Loading the model')
    model = load_model('hist_lstm_model.h5')

    print('Deciding whats hate and what aint')
    prediction_prob = model.predict(padded_tweets)
    prediction = keras_output_sklearn(prediction_prob)

    return [prediction, prediction_prob]

In [9]:
def all_together_for_notebook(tweet_list):
    print('Cleaning tweets')
    cleaned1 = clean_sentences(tweet_list)

    print('Tokenizing tweets')
    tokened_tweets = token_maker(cleaned1)

    print('Padding tweets')
    padded_tweets = sequence.pad_sequences(tokened_tweets, maxlen=len_max)

    print('Loading the model')
    model = load_model('../models/model_acc.h5')

    print('Deciding whats hate and what aint')
    prediction_prob = model.predict(padded_tweets)
    prediction = keras_output_sklearn(prediction_prob)
    
    print('enjoy your results')
    return [prediction, prediction_prob]

In [10]:
def all_together_for_notebook_1(tweet_list):
    print('Cleaning tweets')
    cleaned1 = clean_sentences(tweet_list)

    print('Tokenizing tweets')
    tokened_tweets = token_maker(cleaned1)

    print('Padding tweets')
    padded_tweets = sequence.pad_sequences(tokened_tweets, maxlen=len_max)

    print('Loading the model')
    model = load_model('../models/model_no.h5')

    print('Deciding whats hate and what aint')
    prediction_prob = model.predict(padded_tweets)
    prediction = keras_output_sklearn(prediction_prob)
    
    print('enjoy your results')
    return [prediction, prediction_prob]

In [11]:
y_pred = all_together_for_notebook(tweets)
y_pred_1 = all_together_for_notebook(tweets)

Cleaning tweets


W0802 17:08:37.591905 4413814208 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0802 17:08:37.605420 4413814208 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0802 17:08:37.616581 4413814208 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0802 17:08:37.690769 4413814208 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0802 17:08:37.696987 4413814208 deprecati

Tokenizing tweets
Padding tweets
Loading the model


W0802 17:08:38.219974 4413814208 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0802 17:08:38.384239 4413814208 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0802 17:08:38.485440 4413814208 deprecation.py:323] From /anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Deciding whats hate and what aint
enjoy your results
Cleaning tweets
Tokenizing tweets
Padding tweets
Loading the model
Deciding whats hate and what aint
enjoy your results


In [12]:
len(y_pred[0]) - len(tweets)

0

In [13]:
hate = 0
hurtful = 0
neither = 0

for x in y_pred[0]:
    if str(x) == '0':
        hate += 1
    elif str(x) == '1':
        hurtful += 1
    elif str(x) == '2':
        neither += 1


print(f'Hateful tweets: {hate}; % of total: {hate/(hate+hurtful+neither)}')
hate_results = f'Hateful tweets: {hate}; % of total: {hate/(hate+hurtful+neither)}'

print(f'Hurtful tweets: {hurtful}; % of total: {hurtful/(hate+hurtful+neither)}')
hurtful_results = f'Hurtful tweets: {hurtful}; % of total: {hurtful/(hate+hurtful+neither)}'

print(f'Neither tweets: {neither}; % of total: {neither/(hate+hurtful+neither)}')
neither_results = f'Neither tweets: {neither}; % of total: {neither/(hate+hurtful+neither)}'
results = {'hate':hate_results,
           'hurtful':hurtful_results,
           'neither':neither_results}

print('-----------------')
print(results)

Hateful tweets: 2; % of total: 1.0
Hurtful tweets: 0; % of total: 0.0
Neither tweets: 0; % of total: 0.0
-----------------
{'hate': 'Hateful tweets: 2; % of total: 1.0', 'hurtful': 'Hurtful tweets: 0; % of total: 0.0', 'neither': 'Neither tweets: 0; % of total: 0.0'}


In [14]:
hate = 0
hurtful = 0
neither = 0

for x in y_pred_1[0]:
    if str(x) == '0':
        hate += 1
    elif str(x) == '1':
        hurtful += 1
    elif str(x) == '2':
        neither += 1


print(f'Hateful tweets: {hate}; % of total: {hate/(hate+hurtful+neither)}')
hate_results = f'Hateful tweets: {hate}; % of total: {hate/(hate+hurtful+neither)}'

print(f'Hurtful tweets: {hurtful}; % of total: {hurtful/(hate+hurtful+neither)}')
hurtful_results = f'Hurtful tweets: {hurtful}; % of total: {hurtful/(hate+hurtful+neither)}'

print(f'Neither tweets: {neither}; % of total: {neither/(hate+hurtful+neither)}')
neither_results = f'Neither tweets: {neither}; % of total: {neither/(hate+hurtful+neither)}'
results = {'hate':hate_results,
           'hurtful':hurtful_results,
           'neither':neither_results}

print('-----------------')
print(results)

Hateful tweets: 2; % of total: 1.0
Hurtful tweets: 0; % of total: 0.0
Neither tweets: 0; % of total: 0.0
-----------------
{'hate': 'Hateful tweets: 2; % of total: 1.0', 'hurtful': 'Hurtful tweets: 0; % of total: 0.0', 'neither': 'Neither tweets: 0; % of total: 0.0'}
